<a href="https://colab.research.google.com/github/Ralls0/logocGAN/blob/ral%2Fbup/logocGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import pandas

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from sklearn.utils import shuffle
from imutils import build_montages
from tqdm import tqdm
from datetime import datetime
from google.colab import drive
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow import keras


In [ ]:
# mount google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Environment

In [ ]:
# env
NUM_DIMS=100                      # Number of dimension manipulated into the lateted space 
BATCH_SIZE=64                     #
INIT_LR=2e-4                      # Learning rate
image_size = (256,256)            #
noise = Input((NUM_DIMS))         # 
BATCH_SIZE_2 = BATCH_SIZE // 2
NUM_EPOCHS=300

directory="/content/drive/MyDrive/Politecnico/logo"
base_class="/1"
directory_models="/content/drive/MyDrive/Politecnico/gen/"

# Utils

In [ ]:
# Function to generate random points in the latent space
# in this case, we are using a Gaussian (normal) distribution with 0 mean and 1.
def generate_latent_points(latent_dim, n_samples):

	x_input = np.random.randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [ ]:
# Function to preprocess the images so that they are in the range [-1, 1]
# the range of the real images should be the same of the fake images 

def preprocessing_function(x):
  return (x - 127.5)/127.5

In [ ]:
# Utility function to plot some figures
def plot_figures(x, n, figsize=None):
  if figsize:
      plt.figure(figsize=figsize)
  for i in range(n*n):
      plt.subplot(n,n,i+1)
      plt.xticks([])
      plt.yticks([])
      plt.grid(False)
      img=x[i,:,:,:]
      # rescale for visualization purposes
      img = ((img*127.5) + 127.5).astype("uint8")
      plt.imshow(img)
    
  plt.show()

In [ ]:
# Utility function to store some figures
def store_figures(x, n, directory, epoch):
  for i in range(n*n):
    img=x[i,:,:,:]
    # rescale for visualization purposes
    img = ((img*127.5) + 127.5).astype("uint8")
    cv2.imwrite(os.path.join(directory, "dcgan_img_gen-" + str(epoch) + "-" + str(i) + ".png"), img)

In [ ]:
# Utility function to plot loss figures
def plot_losses(history, lim):
  pandas.DataFrame(history).plot(figsize=(10,8))
  plt.grid(True)
  plt.gca().set_ylim(0,lim)
  plt.show()

# Model definition

In [ ]:
def build_generator(width, height, inputDim=100, n1=2048, channels=3):

  inp = Input(shape=(inputDim,))

  # FC - BN 
  dim1 = width // 16      # because we have 4 transpose conv layers with strides 2 -> 
  dim2 = height // 16     # -> we are upsampling by a factor of 64
  
  x = Dense( dim1 * dim2 * n1, activation="relu")(inp)
  x = BatchNormalization()(x)

  # Reshape to width * heigh * feature_channels

  x = Reshape((dim1, dim2,n1))(x)


  # Conv 2D transpose
  x = Conv2DTranspose(n1//2, (5, 5), strides=(2, 2), padding="same", activation="relu")(x)
  x = BatchNormalization()(x)

  x = Conv2DTranspose(n1//4, (5, 5), strides=(2, 2), padding="same", activation="relu")(x)
  x = BatchNormalization()(x)

  x = Conv2DTranspose(n1//8, (5, 5), strides=(2, 2), padding="same", activation="relu")(x)
  x = BatchNormalization()(x)

  
  # Final layer with tanh activation
  out = Conv2DTranspose(channels, (5,5), strides=(2,2), padding="same", activation="tanh")(x)

  m = Model(inputs=inp, outputs=out)
  return m


In [ ]:
# builds a discriminator with 3 convolutional layers

def build_discriminator(width, height, channels, alpha=0.2, droprate=0.4):
    input_shape = (width, height, channels)

    # use Leaky ReLU instead of Relu in the discriminator
    leaky = tf.keras.layers.LeakyReLU(alpha)
    n1 = 128
    n2 = 256
    n3 = 512

    inp = Input(shape=input_shape)
    x = Conv2D(n1, (5,5), strides=(2,2), activation=leaky)(inp)
    x = Conv2D(n2, (5,5), strides=(2,2), activation=leaky)(x)
    x = Conv2D(n3, (3,3), strides=(2,2), activation=leaky)(x)
   
    x = Flatten()(x)

    # Increase variability in the discriminator by means of `Dropout`
    x = Dropout(droprate)(x)
    out = Dense(1, activation="sigmoid")(x)

    m = Model(inputs=inp, outputs=out)
    return m


# Compiling models

In [ ]:
# Generator
gen = build_generator(image_size[0], image_size[1], inputDim=NUM_DIMS, n1=2048, channels=3)
gen.summary()


In [ ]:
# Discriminator
disc = build_discriminator(image_size[0], image_size[1], 3, droprate=0.5)
disc.summary()


In [ ]:
# Compile the discriminator
discOpt = tf.keras.optimizers.Adam(lr=INIT_LR, beta_1=0.5)                # beta_1 was could achieve best performances with Adam
disc.compile(discOpt, loss="binary_crossentropy", metrics=["accuracy"])
disc.summary()


In [ ]:
# Connecting the two models

# Setting the discriminator to not trainable
disc.trainable = False

# Connecting the discriminator with the generator
discOutput=disc(gen(noise))
gan = Model(inputs=noise, outputs=discOutput)

ganOpt = tf.keras.optimizers.Adam(lr=INIT_LR, beta_1=0.5)
gan.compile(loss="binary_crossentropy", optimizer=ganOpt)


In [ ]:
gan.summary()
disc.summary()


In [ ]:
# noise vector used during training in order to evaluate how the network is learning
benchmarkNoise = generate_latent_points(NUM_DIMS, BATCH_SIZE)

In [ ]:
train_generator = keras.preprocessing.image.ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    preprocessing_function=preprocessing_function
)


In [ ]:
training_size = len(os.listdir(directory+base_class) )
print(f" Traning size: {training_size}")

1940


In [ ]:
train = train_generator.flow_from_directory(directory, target_size=image_size, batch_size=BATCH_SIZE, shuffle=True)

Found 1940 images belonging to 1 classes.


# Training for a given number of epochs



In [ ]:
trueImages, _ = next(train)
trueImages.shape

(64, 256, 256, 3)

In [ ]:
batchesPerEpoch = int(training_size / BATCH_SIZE)
print(f" Batches per epoch: {batchesPerEpoch}")

30


In [ ]:
#keep track of loss and accuracy separately for true and fake images
history = {}
history['G_loss'] = []
history['D_loss_true'] = []
history['D_loss_fake'] = []

accuracy = {}
accuracy['Acc_true'] = []
accuracy['Acc_fake'] = []

# for each epoch
for epoch in range(NUM_EPOCHS):
  batchesPerEpoch = int(training_size / BATCH_SIZE)
  q = 1
  now = datetime.now()
  current_time = now.strftime("%H:%M:%S")
  print(f"[i] ({current_time}) Epoch {epoch} [", end ="")

  # for each batch
  for b in (range(1, batchesPerEpoch+1)):
    if ((b*10/batchesPerEpoch) % q ) == 0:
      q=q+1
      print("=", end ="")

    # Training the discriminator to differentiate between true and fake images
    trueImages, _ = next(train)

    # One sided label smoothing reduces overconfidence in true images and stabilizes training a bit
    y = 0.9*np.ones((trueImages.shape[0]))
    discLoss, discAcc = disc.train_on_batch(trueImages, y)
    
    history['D_loss_true'].append(discLoss)    

    # Warning: accuracy will not be calculated if label smoothing is used 
    accuracy['Acc_true'].append(discAcc)
    
    # Generating some fake samples
    noise =  generate_latent_points(NUM_DIMS, BATCH_SIZE)
    genImages=gen.predict(noise)
    y = np.zeros((BATCH_SIZE))

    discLoss, discAcc = disc.train_on_batch(genImages, y)
    history['D_loss_fake'].append(discLoss)          
    accuracy['Acc_fake'].append(discAcc)

    # Training the generator
    noise =  generate_latent_points(NUM_DIMS, BATCH_SIZE)

    # NOTE: Some authors suggest randomly flipping some labels to introduce random variations
    fake_labels = [1] * BATCH_SIZE
    fake_labels = np.reshape(fake_labels, (-1,))
    ganLoss = gan.train_on_batch(noise, fake_labels)
    
    history['G_loss'].append(ganLoss)

  print("]:", end=" ")
  print(f"Discriminator loss {str(discLoss)} ( {str(discAcc)} ) - Generator loss {str(ganLoss)}")

  images = gen.predict(benchmarkNoise)

  # Visualizing the output
  if (epoch % 10) == 0:
    store_figures(images, 4, directory_models, epoch)
    plot_figures(images, 4)


  # Saving
  if (epoch % 100) == 0:
    gan.save(os.path.join(directory_models, "logocGAN_" + str(epoch)+".h5"))


# Plotting metrics

In [ ]:
plot_losses(history, 3)
plot_losses(accuracy, 1)
plot_figures(trueImages, 4, figsize=(10, 10))

In [ ]:
# Reloading the model at the selecte epoch

epoch = 200
leaky = tf.keras.layers.LeakyReLU(0.2)
tf.keras.utils.get_custom_objects().update({'LeakyReLU': leaky})
gan = tf.keras.models.load_model(os.path.join(directory_models, "dcgan_2" + str(epoch)+".h5"))

gen = gan.layers[1]

gen.summary()

In [ ]:
noisevector = generate_latent_points(NUM_DIMS, BATCH_SIZE)

genImages = gen.predict(noisevector)    
plot_figures(genImages,4, figsize=(10, 10))